In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(dplyr)

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union




In [4]:
%%R
data=read.csv("/content/NSSO68.csv")

In [15]:
%%R
unique(data$state_1)
# Subset data to state Gujarat
subset_data <- data %>%
  filter(state_1 == 'GUJ') %>%
  select(foodtotal_q, MPCE_MRP, MPCE_URP,Age,Meals_At_Home,Possess_ration_card,Education, No_of_Meals_per_day)
print(subset_data)

     foodtotal_q MPCE_MRP MPCE_URP Age Meals_At_Home Possess_ration_card
1      30.942394  3662.65  3304.80  50            59                   1
2      29.286153  5624.51  7613.00  40            56                   1
3      31.527046  3657.18  3461.40  45            60                   1
4      27.834607  3260.37  3339.00  75            60                   1
5      27.600713  2627.54  2604.25  30            59                   1
6      38.000992  3788.63  4391.00  45            60                   1
7      18.710805  2122.85  2107.80  34            60                   1
8      16.417975  1005.96   953.25  37            60                   1
9      49.100800  3600.70  3447.00  52            52                   1
10     37.975780  3940.34  4396.50  45            60                   1
11     16.650159  1448.45  1323.00  46            60                   1
12     17.266835  1327.32  1364.83  51            60                   1
13     19.485282  1312.42  1205.80  62            6

In [16]:
%%R
sum(is.na(subset_data$MPCE_MRP))
sum(is.na(subset_data$MPCE_URP))
sum(is.na(subset_data$Age))
sum(is.na(subset_data$Possess_ration_card))
sum(is.na(data$Education))


[1] 0


In [17]:
%%R
impute_with_mean <- function(data, columns) {
  data %>%
    mutate(across(all_of(columns), ~ ifelse(is.na(.), mean(., na.rm = TRUE), .)))
}


In [18]:
%%R
# Columns to impute
columns_to_impute <- c("Education")

# Impute missing values with mean
data <- impute_with_mean(data, columns_to_impute)

sum(is.na(data$Education))

[1] 0


In [20]:
%%R
# Fit the regression model
model <- lm(MPCE_MRP ~ MPCE_URP + Age + Meals_At_Home + Possess_ration_card + Education + foodtotal_q, data = subset_data)
summary(model)


Call:
lm(formula = MPCE_MRP ~ MPCE_URP + Age + Meals_At_Home + Possess_ration_card + 
    Education + foodtotal_q, data = subset_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-9906.9  -470.2  -157.8   233.7 18163.8 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)         625.86295  272.38391   2.298   0.0217 *  
MPCE_URP              0.48336    0.01794  26.936  < 2e-16 ***
Age                   1.17195    2.74807   0.426   0.6698    
Meals_At_Home       -27.46606    3.32660  -8.257  3.2e-16 ***
Possess_ration_card  41.62641   92.10420   0.452   0.6514    
Education           110.00084   11.64271   9.448  < 2e-16 ***
foodtotal_q          59.42516    4.90323  12.120  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1386 on 1532 degrees of freedom
Multiple R-squared:  0.5539,	Adjusted R-squared:  0.5521 
F-statistic:   317 on 6 and 1532 DF,  p-value: < 2.2e-16



In [21]:
%%R
install.packages("car")
library(car)
# Check for multicollinearity using Variance Inflation Factor (VIF)
vif(model) # VIF Value more than 8 its problematic

(as ‘lib’ is unspecified)







	‘/tmp/RtmpXO5FHY/downloaded_packages’



           MPCE_URP                 Age       Meals_At_Home Possess_ration_card 
           1.339350            1.142515            1.349263            1.116431 
          Education         foodtotal_q 
           1.089335            1.511599 


In [22]:
%%R
# Extract the coefficients from the model
coefficients <- coef(model)

# Construct the equation
equation <- paste0("y = ", round(coefficients[1], 2))
for (i in 2:length(coefficients)) {
  equation <- paste0(equation, " + ", round(coefficients[i], 6), "*x", i-1)
}
# Print the equation
print(equation)

[1] "y = 625.86 + 0.483358*x1 + 1.171946*x2 + -27.466057*x3 + 41.626412*x4 + 110.000843*x5 + 59.425163*x6"


In [30]:
%%R

print(head(subset_data$MPCE_MRP,1))
print(head(subset_data$MPCE_URP,1))
print(head(subset_data$Age,1))
print(head(subset_data$Meals_At_Home,1))
print(head(subset_data$Possess_ration_card,1))
print(head(subset_data$Education,1))
print(head(subset_data$foodtotal_q,1))


[1] 3662.65
[1] 3304.8
[1] 50
[1] 59
[1] 1
[1] 8
[1] 30.94239
